In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import make_scorer, accuracy_score, f1_score

# Cargar el conjunto de datos
file_path = 'dataset.csv'
data = pd.read_csv(file_path)

# exploramos los datos
print(data.describe())
sns.pairplot(data, hue='y', diag_kind='kde')
plt.show()

# normalizamos
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(data.iloc[:, :-1])
normalized_data = pd.DataFrame(scaled_features, columns=data.columns[:-1])
normalized_data['y'] = data['y']

# Configurar validación cruzada k-fold
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# --- Implementación de kNN ---
k_values = [1, 3, 5, 7, 9]
k_scores = {"k": [], "accuracy": [], "f1": []}

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    accuracy = cross_val_score(knn, normalized_data.iloc[:, :-1], normalized_data['y'], 
                                cv=kfold, scoring=make_scorer(accuracy_score)).mean()
    f1 = cross_val_score(knn, normalized_data.iloc[:, :-1], normalized_data['y'], 
                         cv=kfold, scoring=make_scorer(f1_score, pos_label='A')).mean()
    k_scores["k"].append(k)
    k_scores["accuracy"].append(accuracy)
    k_scores["f1"].append(f1)

knn_results = pd.DataFrame(k_scores)
print("Resultados de kNN:")
print(knn_results)

# --- Implementación de ANN ---
hidden_layer_sizes = [5, 10, 15, 20]
ann_scores = {"neurons": [], "accuracy": [], "f1": []}

for size in hidden_layer_sizes:
    ann = MLPClassifier(hidden_layer_sizes=(size,), max_iter=1000, random_state=42)
    accuracy = cross_val_score(ann, normalized_data.iloc[:, :-1], normalized_data['y'], 
                                cv=kfold, scoring=make_scorer(accuracy_score)).mean()
    f1 = cross_val_score(ann, normalized_data.iloc[:, :-1], normalized_data['y'], 
                         cv=kfold, scoring=make_scorer(f1_score, pos_label='A')).mean()
    ann_scores["neurons"].append(size)
    ann_scores["accuracy"].append(accuracy)
    ann_scores["f1"].append(f1)

ann_results = pd.DataFrame(ann_scores)
print("Resultados de ANN:")
print(ann_results)

# --- Comparación de resultados ---
plt.figure(figsize=(10, 5))

# Gráfico para kNN
plt.subplot(1, 2, 1)
sns.lineplot(x=knn_results["k"], y=knn_results["accuracy"], label="Accuracy", marker='o')
sns.lineplot(x=knn_results["k"], y=knn_results["f1"], label="F1-Score", marker='o')
plt.title("kNN Performance")
plt.xlabel("k (número de vecinos)")
plt.ylabel("Score")
plt.legend()

# Gráfico para ANN
plt.subplot(1, 2, 2)
sns.lineplot(x=ann_results["neurons"], y=ann_results["accuracy"], label="Accuracy", marker='o')
sns.lineplot(x=ann_results["neurons"], y=ann_results["f1"], label="F1-Score", marker='o')
plt.title("ANN Performance")
plt.xlabel("Neurons (capa oculta)")
plt.ylabel("Score")
plt.legend()

plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'pandas'